In [ ]:
!pip3 install transformers>=4.32.0 optimum>=1.12.0
!pip3 install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/  # Use cu117 if on CUDA 11.7

Looking in indexes: https://pypi.org/simple, https://huggingface.github.io/autogptq-index/whl/cu118/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 9.6 MB/s eta 0:00:00


In [ ]:
from pathlib import Path
import pandas as pd
from tqdm import tqdm
import os
import torch

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

## Read Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
BASE_DIR = Path("/content/drive/MyDrive/ITMO/RW'23")
DATA_DIR = BASE_DIR.joinpath("data")

FILE_NAME = "data_quality_estim.csv"

FILE = DATA_DIR.joinpath(FILE_NAME)

In [ ]:
data = pd.read_csv(FILE)
data.head(10)

,task,agg_label
0,able amaze fill glad stop daily fantastic move...,bad
1,able amaze fill stop glad daily fantastic move...,bad
2,able amaze fill stop glad daily fantastic move...,bad
3,able amaze glad fill stop fantastic daily move...,bad
4,absolute church truth doug scripture symbol li...,good
5,absolute church truth mission nasa doug mars e...,bad
6,absolute mission church nasa mars earth truth ...,good
7,absolutely fast white rice service noodle terr...,bad
8,absolutely fast white super service customer r...,good
9,absolutely grain experience soon there's proce...,bad


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3861 entries, 0 to 3860
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   task       3861 non-null   object
 1   agg_label  3861 non-null   object
dtypes: object(2)
memory usage: 60.5+ KB


##Llama-13B

In [ ]:
!nvidia-smi

Thu Jan 11 19:11:25 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GPTQ"
# To use a different branch, change revision
# For example: revision="main"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/837 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/7.26G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

##Inference

In [ ]:
def generate_answer(prompt, model, device, temp, n_token):
    encoded = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    model_input = encoded
    model_input = model_input.to(device)
    generated_ids = model.generate(
        **model_input, do_sample=True,
        max_new_tokens=n_token,
        temperature=temp,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,
        num_beams=4
        )
    decoded = tokenizer.batch_decode(generated_ids)

    return decoded

In [ ]:
def extract_substring(input_string):
    index = input_string.find("[/INST]")
    if index != -1:
        return input_string[index + len("[/INST]"):]
    else:
        return ""

In [ ]:
def few_shot_pipeline(model, system_prompt='', instruction='', sample='', device='cpu', temp=0.1, n_token=100):
    # Construct prompt
    prompt = f"""<s>[INST]<<SYS>> {system_prompt} <</SYS>> {instruction} TEXT: {sample} ANSWER: [/INST]"""
    # Generate answer
    answer = generate_answer(prompt, model, device, temp, n_token)
    # Extract the result
    # print(answer[0])
    result = extract_substring(answer[0]).strip()
    return result


[INST] << SYS >>{system_prompt} << /SYS >>
{prompt}[/INST]

####Type I

In [ ]:
prompt = """<s>[INST] <<SYS>> You are the assistant for text classification <<SYS>>
You will receive a TEXT, and you should answer "YES" or "NO" to the question: "Is it possible to determine a common topic for the TEXT or at least for the most part of the TEXT?". Please, make sure you to only return YES or NO and nothing more.
TEXT: april nasa star canada bike score always goal maynard area player hockey wings center white
ANSWER:[/INST]""".strip()

answer = generate_answer(prompt, model, 'cuda', temp=0.1, n_token=100)

print(answer[0])


<s>[INST] <<SYS>> You are the assistant for text classification <<SYS>>
You will receive a TEXT, and you should answer "YES" or "NO" to the question: "Is it possible to determine a common topic for the TEXT or at least for the most part of the TEXT?". Please, make sure you to only return YES or NO and nothing more.
TEXT: april nasa star canada bike score always goal maynard area player hockey wings center white
ANSWER:[/INST]  NO.</s>


### Low-level code for text generation

In [ ]:
prompt = """<<SYS>> You are the assistant for text classification <</SYS>>
You will receive a TEXT, and you should answer "YES" or "NO" to the question: "Is it possible to determine a common topic for the TEXT or at least for the most part of the TEXT?". Please, make sure you to only return YES or NO and nothing more.

TEXT: april nasa star canada bike score always goal maynard area player hockey wings center white
ANSWER:""".strip()

print(prompt)

voc = tokenizer.get_vocab()
voc_rev = {v:k for k, v in voc.items()}

for i in range(1):
    inputs = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False).to('cuda')
    logits = model.forward(**inputs).logits[0, -1, :]
    probs = torch.nn.functional.softmax(logits, dim=-1)
    next_token_id = torch.multinomial(probs.flatten(), num_samples=1)

    next_token = tokenizer.decode(next_token_id)
    print("next token", next_token)
    prompt += next_token


    sorted_probs, sorted_indices = torch.sort(probs, descending=True)
    top_tokens = sorted_indices[:5]
    print(f'Step #{i} candidates:')
    for t, p in zip(top_tokens, sorted_probs):
        t = voc_rev[t.item()]
        print(f"{t}: {p:.4f} ")

    print(f"\nChosen token: (next_token)", end="\n\n", flush=True)

<<SYS>> You are the assistant for text classification <</SYS>>
You will receive a TEXT, and you should answer "YES" or "NO" to the question: "Is it possible to determine a common topic for the TEXT or at least for the most part of the TEXT?". Please, make sure you to only return YES or NO and nothing more.

TEXT: april nasa star canada bike score always goal maynard area player hockey wings center white
ANSWER:
next token YES
Step #0 candidates:
▁YES: 0.3444 
▁NO: 0.2900 
<0x0A>: 0.2224 
▁: 0.0441 
▁?: 0.0169 

Chosen token: (next_token)



####Type II

In [ ]:
test_data = data.sample(frac=1, random_state=42)

In [ ]:
test_data

,task,agg_label
2424,pack variety com www http href stash pie grey ...,bad
3290,south island ship secret military rutgers brit...,bad
2575,piece beef jerky everything season break pill ...,bad
2042,mix cream ice pour strawberry stir pancake waf...,good
3717,window display screen color server application...,good
...,...,...
1130,engine model rocket band lock foot exhaust pre...,good
1294,font print character page printer vice cell bi...,good
860,delicious small dry hard smaller surprised liv...,bad
3507,thought end else stick cover boil air china bu...,bad


In [ ]:
test_data.iloc[1795]

task         launch entry satellite rule remark build europ...
agg_label                                                  bad
Name: 1872, dtype: object

In [ ]:
SYSTEM_PROMPT = "You are the assistant for text classification"
INSTRUCTION = "You will receive a TEXT, and you should answer 'YES' or 'NO' to the question: 'Is it possible to determine a common topic for the TEXT or at least for the most part of the TEXT?'. Please, make sure you to only return YES or NO and nothing more."

SAMPLE = test_data.iloc[785]["task"]
print(SAMPLE)

res = few_shot_pipeline(model, SYSTEM_PROMPT, INSTRUCTION, SAMPLE, device="cuda", n_token=30)

res

firearm weapon bill crime vote public laws handgun defense criminal washington federal member carry unite


'NO. There is no common topic that can be determined for the majority of the text. The text mentions various topics such as firearms,'

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# Create file
RES_FILE_NAME = "data_qality_estim_llama_13b_4bit_type1.csv"
RES_FILE = DATA_DIR.joinpath(RES_FILE_NAME)
# # Remove old file
# if os.path.exists(RES_FILE):
#     os.remove(RES_FILE)
for i in tqdm(range(785, len(test_data))):
# for i in tqdm(range(20)):
    SAMPLE = test_data.iloc[i]["task"]
    res_true = test_data.iloc[i]["agg_label"]
    res_model = few_shot_pipeline(
        model,
        SYSTEM_PROMPT,
        INSTRUCTION,
        SAMPLE,
        device="cuda",
        temp=0.1,
        n_token=30
    )
    with open(RES_FILE, 'a') as file:
        file.write(f"{SAMPLE};{res_true};{res_model}\n")


100%|██████████| 3076/3076 [1:26:57<00:00,  1.70s/it]


##Type V Entire Dataset

In [ ]:
INSTRUCTION = """Select one of two categories-characteristics of the provided set of tokens by answering the question ”Is it possible to determine a common topic for the presented word set or at least for the most part of the set?”.
You could answer one of the following: YES - if you agree with the statement and words have a strong connection between them, NO - when words seem to be unconnected. Please, make sure you to only return YES or NO and nothing more.

bike firearm knife criminal carry death weapon scsi murder smith handgun moral child defense tape // YES
appreciate advance anybody info thank experience hello greatly suggestion wonder edge able email recently thanx // NO
bible christ matthew heaven scripture lord father word passage teach mary spirit doctrine verse faith // YES"""

SAMPLE = data.iloc[80]["task"]

res = few_shot_pipeline(INSTRUCTION, SAMPLE, model, device="cuda", n_token=100)

res

' YES</s>'

In [ ]:
test_data = data.sample(frac=1, random_state=42)

In [ ]:
test_data

,task,agg_label
2424,pack variety com www http href stash pie grey ...,bad
3290,south island ship secret military rutgers brit...,bad
2575,piece beef jerky everything season break pill ...,bad
2042,mix cream ice pour strawberry stir pancake waf...,good
3717,window display screen color server application...,good
...,...,...
1130,engine model rocket band lock foot exhaust pre...,good
1294,font print character page printer vice cell bi...,good
860,delicious small dry hard smaller surprised liv...,bad
3507,thought end else stick cover boil air china bu...,bad


In [ ]:
# Create file
RES_FILE_NAME = "data_qality_estim_mistral_7b_4bit_type5.csv"
RES_FILE = DATA_DIR.joinpath(RES_FILE_NAME)
# # Remove old file
# if os.path.exists(RES_FILE):
#     os.remove(RES_FILE)
with open(RES_FILE, 'a') as file:
    for i in tqdm(range(len(test_data))):
    # for i in tqdm(range(100)):
        SAMPLE = test_data.iloc[i]["task"]
        res_true = test_data.iloc[i]["agg_label"]
        res_model = few_shot_pipeline(
            INSTRUCTION,
            SAMPLE,
            model,
            device="cuda",
            temp=0.1,
            n_token=100
        )
        file.write(f"{SAMPLE};{res_true};{res_model}\n")


100%|██████████| 3861/3861 [2:04:51<00:00,  1.94s/it]


##Type VI Entire Dataset

In [ ]:
INSTRUCTION = """You are a human. Select one of two categories-characteristics of the provided set of tokens by answering the question ”Is it possible to determine a common topic for the presented word set or at least for the most part of the set?”.
You could answer one of the following: YES - if you agree with the statement and words have a strong connection between them, NO - when words seem to be unconnected. Please, make sure you to only return YES or NO and nothing more.

bike firearm knife criminal carry death weapon scsi murder smith handgun moral child defense tape // YES
appreciate advance anybody info thank experience hello greatly suggestion wonder edge able email recently thanx // NO
bible christ matthew heaven scripture lord father word passage teach mary spirit doctrine verse faith // YES"""

SAMPLE = data.iloc[0]["task"]

res = few_shot_pipeline(INSTRUCTION, SAMPLE, model, device="cuda", n_token=100)

res

' YES</s>'

In [ ]:
test_data = data.sample(frac=1, random_state=42)

In [ ]:
# Create file
# При записи в файл убирать знаки переноса на новую строку и ";"
RES_FILE_NAME = "data_qality_estim_mistral_7b_4bit_type6.csv"
RES_FILE = DATA_DIR.joinpath(RES_FILE_NAME)
# Remove old file
if os.path.exists(RES_FILE):
    os.remove(RES_FILE)
with open(RES_FILE, 'a') as file:
    for i in tqdm(range(len(test_data))):
    # for i in tqdm(range(100)):
        SAMPLE = test_data.iloc[i]["task"]
        res_true = test_data.iloc[i]["agg_label"]
        res_model = few_shot_pipeline(
            INSTRUCTION,
            SAMPLE,
            model,
            device="cuda",
            temp=0.1,
            n_token=100
        )
        file.write(f"{SAMPLE};{res_true};{res_model}\n")


 94%|█████████▍| 3635/3861 [1:58:47<07:31,  2.00s/it]